**Outline**

The stimulus channel for the buttonPress data recieves two signals (i.e. amplitude peaks) per trial - the auditory cue (low amplitude) and the buttonPress (high amplitude). In a later script, we automatically detect events (button presses) based on some threshold below the BP amplitude, but above the cue amplitude. It turns out that ampltiude peaks are not consistent between participants, meaning that one threshold does not fit everybody. Therefore, the purpose of this script is to determine an appropriate threshold that detects BP's and ignores cues, for each participant. 

For each participant, we'll visualize events detected at a particular threshold, overlaid on the stimulus channel. We can also identify bad events (e.g., un-cued button presses). Thresholds and bad events are logged in the buttonPress_stimChannel_threshold_and_bad_events.txt file, in the support_files directory

**Import packages**

In [ ]:
import mne
import os
import numpy as np
import scipy.signal as ssig
import matplotlib.pyplot as plt 
from ipywidgets import *
import json

**Define data paths, subject, task prefix, and threshold**

In [ ]:
# Define path containing raw data
projectDir = '../'
dataDir = os.path.join(projectDir, 'proc_data')
supportDir = os.path.join(projectDir, 'support_files')

# Define raw data file
rawPre = 'buttonPress'
rawFile = f'{rawPre}-trans-raw.fif'

# Define events file
eventsFile = os.path.join(supportDir, f'{rawPre}_stimChannel_threshold_and_bad_events.txt')

# Set threshold for event detection
vThresh = 0.5

**Load the raw data, detect events, and plot**

In [ ]:
# Select a subject
subjectID = 'mnsbp001'

# Read raw data for this subject
raw = mne.io.read_raw(os.path.join(dataDir, subjectID, 'meg', rawFile))

# Get stim channel from the raw file
stimData = raw.get_data()[-1,:]
stimDiff = np.diff(stimData)

# Find peaks. Use stimDiff to find event onsets, or -stimDiff to find event offsets.
bpIndex = ssig.find_peaks(-stimDiff, vThresh, distance=100)[0] 

print(f'{len(bpIndex)} events found')

# Plot events
%matplotlib widget
fig, ax = plt.subplots(1,1, figsize=(10,5))
ax.plot(stimData, 'k', linewidth=0.5)
for n, i in enumerate(bpIndex):

    # Plot the event
    ax.axvline(i, linestyle='--', linewidth=0.75)

    # Add the event number
    ax.text(i, np.max(stimData), str(n), fontsize=8)
    
plt.show()


**Verify that thresholds and events recorded in the thresholds/bad events file are correct**

In [ ]:
# Create a dict containing events thresholds and bad events for all subjects, based on the events file
event_thresholds_and_bads = {}

with open(eventsFile, "r") as file:
    for line in file:
        line = line.strip()
        if not line or line.startswith("#"):  # Skip comments and empty lines
            continue

        # Split the line at the '=' and remove any trailing comments
        key, value = line.split("=", 1)  # Only split on the first '='
        key = key.strip()

        # Remove any comments from the value part (after '#')
        value = value.split("#")[0].strip()

        # Append key and value to the dict
        event_thresholds_and_bads[key] = json.loads(value)  # json.loads parses value (a string) into a list

In [ ]:
threshold, bads = event_thresholds_and_bads[subjectID]

print(f'Threshold: {threshold}')
print(f'Bad events: {bads}')